The following query adapts [this example](https://colab.research.google.com/drive/1HhEEB3MJ8LbMP2ta946s8OARPc5RflHu?usp=sharing#scrollTo=nM-GHd3IWeqF) from the January 2021 FHIR Connectathon on Bulk-FHIR.


The query looks for patients whose latest observation for HP:0410030 (abnormal facial features) was positive.

Note the following is on the DNAStack staging server

In [23]:
from fasp.search import DiscoverySearchClient
cl = DiscoverySearchClient('http://ga4gh-search-adapter-presto-public.staging.dnastack.com')

query = """SELECT * FROM (
    SELECT 
      json_extract_scalar(Observation.observation, '$.subject.reference') patient,
      json_extract_scalar(Observation.observation, '$.extension[0].valueAge.value') ageAtEvent,
      json_extract_scalar(Observation.observation, '$.code.text') phenotype,
      json_extract_scalar(Observation.observation, '$.interpretation[0].coding[0].code') observed,
      row_number() OVER (
        PARTITION BY    
          json_extract_scalar(Observation.observation, '$.subject.reference')
        ORDER BY
          json_extract_scalar(Observation.observation, '$.extension[0].valueAge.value') DESC
      ) as rank
    FROM
      kidsfirst.ga4gh_tables.observation,
      UNNEST (cast(json_extract(Observation.observation, '$.code.coding') as array(json))) AS coding(value)
      WHERE
        json_extract_scalar(coding.value, '$.system') = 'http://purl.obolibrary.org/obo/hp.owl'
        AND json_extract_scalar(coding.value, '$.code') = 'HP:0410030'
  )
  WHERE rank = 1 AND observed = 'POS'
  """

df = cl.runQuery(query, returnType='dataframe')
df

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________
____Page8_______________
____Page9_______________
____Page10_______________
____Page11_______________
____Page12_______________


,patient,ageAtEvent,phenotype,observed,rank
0,Patient/451886,2816,Cleft Lip,POS,1
1,Patient/452918,472,Cleft Lip,POS,1
2,Patient/453228,25,Cleft Lip,POS,1
3,Patient/451828,3055,Cleft Lip,POS,1
4,Patient/453121,20,Cleft Lip,POS,1


Taking it a step further, we can use the HPO ontology to expand the query to find all patients who have any abnormality of the face (HP:0410030 and all its subclasses in the HPO ontology).

For performance reasons this was broken into two parts.

First a query to expand the list of HPO terms for all sub-types of facial abnormality.

In [22]:
 termQuery = '''WITH hpo_subclassof(node) as (
   select array_agg(array [REPLACE(from_term,'_',':'), REPLACE(to_term,'_',':')]) from search_postgres_pgpc.ontology.axiom where ontology_version = 'http://purl.obolibrary.org/obo/hp/releases/2019-04-15' and relation='SubClassOf'
 )
 select transitive_closure((SELECT node FROM hpo_subclassof), array ['HP:0410030'])'''
termRes= cl.runQuery(termQuery)
termList = res2[0][0]
termList

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________


['HP:0000153',
 'HP:0000163',
 'HP:0000152',
 'HP:0000001',
 'HP:0031816',
 'HP:0410030',
 'HP:0000271',
 'HP:0000118',
 'HP:0000234',
 'HP:0000202']

That list can then be used in the following query.

In [21]:
query3= '''WITH hpo_subclassof(node) as (
  select array_agg(array [REPLACE(from_term,'_',':'), REPLACE(to_term,'_',':')]) from search_postgres_pgpc.ontology.axiom where ontology_version = 'http://purl.obolibrary.org/obo/hp/releases/2019-04-15' and relation='SubClassOf'
)
SELECT * FROM (
    SELECT 
      json_extract_scalar(Observation.observation, '$.subject.reference') patient,
      json_extract_scalar(Observation.observation, '$.extension[0].valueAge.value') ageAtEvent,
      json_extract_scalar(Observation.observation, '$.code.text') phenotype,
      json_extract_scalar(Observation.observation, '$.interpretation[0].coding[0].code') observed,
      row_number() OVER (
        PARTITION BY    
          json_extract_scalar(Observation.observation, '$.subject.reference')
        ORDER BY
          json_extract_scalar(Observation.observation, '$.extension[0].valueAge.value') DESC
      ) as rank
    FROM
      kidsfirst.ga4gh_tables.observation,
      UNNEST (cast(json_extract(Observation.observation, '$.code.coding') as array(json))) AS coding(value)
      WHERE
        json_extract_scalar(coding.value, '$.system') = 'http://purl.obolibrary.org/obo/hp.owl'
        AND contains(array {}, json_extract_scalar(coding.value, '$.code'))
  )
  WHERE rank = 1 AND observed = 'POS'
  '''.format(termList)
    
res3= cl.runQuery(query3, returnType='dataframe')
res3

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________
____Page8_______________
____Page9_______________
____Page10_______________
____Page11_______________
____Page12_______________
____Page13_______________
____Page14_______________
____Page15_______________
____Page16_______________
____Page17_______________
____Page18_______________
____Page19_______________


,patient,ageAtEvent,phenotype,observed,rank
0,Patient/451173,121,Dysmorphic Facies,POS,1
1,Patient/451415,190,Dysmorphic Facies,POS,1
2,Patient/451743,121,Dysmorphic Facies,POS,1
3,Patient/452493,4866,Dysmorphic Facies,POS,1
4,Patient/452623,26,Dysmorphic Facies,POS,1
5,Patient/452918,472,Dysmorphic Facies,POS,1
6,Patient/453228,25,Dysmorphic Facies,POS,1
7,Patient/451820,904,Dysmorphic Facies,POS,1
8,Patient/452460,34,Dysmorphic Facies,POS,1
9,Patient/452726,369,Dysmorphic Facies,POS,1


In [ ]:
A follow on question from this 